<a href="https://colab.research.google.com/github/RohithJ11/NLP_Privacy_Policies/blob/main/T5updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets torch


  Using cached datasets-2.19.0-py3-none-any.whl (542 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using ca

In [2]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.3 MB/s eta 0:00:00


In [3]:
pip install transformers[torch]

In [4]:
from transformers import Trainer


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch

def preprocess_function(examples):
    # Adjust max_length and max_target_length as needed
    max_length = 512  # Maximum length of the input text
    max_target_length = 128  # Maximum length of the summary text

    # Tokenize the inputs and labels
    model_inputs = tokenizer(examples['Content'], max_length=max_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['Summary_of_Content'], max_length=max_target_length, truncation=True, padding="max_length")

    # T5 expects a -100 in the places where padding tokens appear in labels
    labels['input_ids'] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels['input_ids']
    ]

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


# Load the dataset from Hugging Face Hub
dataset = load_dataset("RohithJ/cleaned_dataset_prvy_plcy")

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocess the data
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./t5_finetuned')

# Example of using the model for summarization
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0])

# Test the model with a sample text from your dataset
sample_text = dataset['test'][0]['Content']
print("Summary:", summarize(sample_text))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,No log,4.727074
2,No log,4.541240
3,No log,4.468998


Summary: <pad> Slack collects, generates and/or receives the following categories of Other Information: Workspace and account information. Slack logs the Workspaces, channels, people, features, content, and links you view or interact with, the types of files shared and what Third-Party Services are used (if any) Log data.</s>


In [12]:
# Specify the directory where you want to save your model
model_save_path = '/content/drive/MyDrive/trained_model/T5model2'

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('/content/drive/MyDrive/trained_model/T5model2/tokenizer_config.json',
 '/content/drive/MyDrive/trained_model/T5model2/special_tokens_map.json',
 '/content/drive/MyDrive/trained_model/T5model2/spiece.model',
 '/content/drive/MyDrive/trained_model/T5model2/added_tokens.json')

In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model and tokenizer
model_path = '/content/drive/MyDrive/trained_model/T5model2'  # Adjust path if your model is saved elsewhere
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
tokenizer = T5Tokenizer.from_pretrained(model_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
def generate_summary(input_text, max_length=3000, num_beams=4):
    # Encode the input text to tensor
    input_ids = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=1024, truncation=True).to(device)

    # Generate summary output ids
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=600, num_beams=num_beams, early_stopping=True)

    # Decode the output ids to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary


In [15]:
pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.1 MB/s eta 0:00:00


In [30]:
import re

def clean_text(text):
    # Remove all double quotes
    text = text.replace('"', '')
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text


In [41]:
import pdfplumber
import re

def clean_text(text):
    # Replace double spaces with single spaces
    text = re.sub(r'\s+', ' ', text)
    # Replace double quotes with single quotes
    text = re.sub(r'\"+', '"', text)
    return text

def extract_text_from_pdf(pdf_path):
    all_text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  # Ensure there's text on the page
                all_text += text + ' '  # Add a space between texts from different pages

    cleaned_text = clean_text(all_text)
    return cleaned_text

# Example usage
pdf_path = '/content/Privacy Policy _ Epson US.pdf'
only_text = extract_text_from_pdf(pdf_path)
print(only_text)


3/31/24, 10:42 AM Privacy Policy | Epson US ___ Epson Privacy Policy United States and Canada Important Information: Please read this Privacy Policy carefully as we have recently updated it. Effective Date: June 30, 2023 Epson's privacy policy provides information about our privacy and data collection practices as well as your rights in relation to your personal information. Table of Contents Introduction Information Collection Use of Information Disclosure of Personal Information Social Media and Technology Integration Network Advertising and Retargeting Your Communication Preferences, Rights and Choices Sweepstakes, Contests and Promotions Children's Privacy Protection The Epson Store Data Security Disability Accommodations Data Relating to Data Subjects in the United Kingdom / European Economic Area / Switzerland Changes to this Policy Job Applicants’ Privacy Statement Your U.S. State Privacy Notice and Rights Notice of Data Practices https://epson.com/privacy-policy 1/71 3/31/24, 1

In [42]:
# Example text
text = only_text

# Generate the summary
summary = generate_summary(text)
print("Summary:", summary)
#print(only_text)

Summary: this Privacy Policy applies only to Epson America, Inc. and Epson Canada, Limited ("epson," "we", "our" or "us") websites where this Privacy Policy is posted (collectively, the "Sites"); other interactive features or downloads that are owned or controlled by Epson and made available through the Sites. if a revised version requires notice under applicable law, we may notify you by other means, such as directly sending you a notification, or as required by applicable law. if....... to epson,... and the. epson's. the 'we, or. ) this Privacy policy. 'Pri......... and other interactive features or.. and that., or that. the "Services", the. and. the. the.............................. our..........................................................................................................................................................................
